In [1]:
import pandas as pd

feedback= pd.read_csv("assets/data/feedback_data.csv")
benefits= pd.read_csv("assets/data/benefits_data.csv")

def clean(df):
    print(df.shape)
    print(df.columns)
    print(df.duplicated().sum())
    df.drop_duplicates(inplace=True)
    print(df.shape)
    print(df.isna().sum())

clean(feedback)
clean(benefits)

(30000, 4)
Index(['EmployeeID', 'BenefitID', 'SatisfactionScore', 'Comments'], dtype='object')
624
(29376, 4)
EmployeeID           0
BenefitID            0
SatisfactionScore    0
Comments             0
dtype: int64
(30, 4)
Index(['BenefitID', 'BenefitType', 'BenefitSubType', 'BenefitCost'], dtype='object')
0
(30, 4)
BenefitID         0
BenefitType       0
BenefitSubType    0
BenefitCost       0
dtype: int64


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
!pip install -qU \
            langchain-openai==0.3.3 \
            langchain==0.3.17 \
            textblob

In [3]:
with open('api.txt', 'r') as file:
    OPENAI_API_KEY = file.read().strip()


In [4]:
df = pd.merge(
    feedback,
    benefits[['BenefitID', 'BenefitType', 'BenefitSubType']],
    on='BenefitID',
    how='left'
)

clean(df)

(29376, 6)
Index(['EmployeeID', 'BenefitID', 'SatisfactionScore', 'Comments',
       'BenefitType', 'BenefitSubType'],
      dtype='object')
0
(29376, 6)
EmployeeID           0
BenefitID            0
SatisfactionScore    0
Comments             0
BenefitType          0
BenefitSubType       0
dtype: int64


In [5]:
print(f"BenefitTypes: {df['BenefitType'].unique()}")
print(f"BenefitSubTypes: {df['BenefitSubType'].unique()}")

BenefitTypes: ['Gym Membership' 'Tuition Reimbursement' 'Commuter Benefits'
 'Life Insurance' 'Retirement Plan' 'Health Insurance'
 'Cell Phone Allowance' 'Flexible Spending Account' 'Wellness Programs'
 'Childcare' 'Professional Development' 'Technology Stipend']
BenefitSubTypes: ['Tier 2 Partners' 'Undergraduate Degree' 'Transit Subsidy'
 'Supplemental Standard' '401k Basic Matching' '401k Investment Fees'
 'Individual Courses' 'Dependent Coverage' 'Basic Coverage'
 'Tier 3 Partners' 'HMO Family' 'PPO Family' '401k Catch-Up Contributions'
 'PPO Individual' 'Monthly Communications' 'HDHP Individual'
 'Healthcare FSA' 'Premium Discount Tier 1' 'After-School Care'
 'Graduate Degree' 'On-Site Infant Care' 'Conference Attendance'
 'Family Membership' 'Monthly Internet Allowance' 'Tier 1 Partners'
 '401k Standard Matching' 'Professional Certification'
 'Supplemental High Amount' '401k Maximum Matching'
 '401k High Contribution']


In [6]:
import os
import pandas as pd
from tqdm import tqdm
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from textblob import TextBlob


# LangChain setup
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini", api_key=OPENAI_API_KEY)
parser = StrOutputParser()

# === Load Feedback Data ===
df = df.head(25)  # limit for testing

# === Step 1: Feedback Classification ===
classify_prompt = PromptTemplate.from_template("""
Classify the following employee comment into **one of the categories** below.
Also consider specific benefit types and subtypes mentioned, such as Gym Membership, Tuition Reimbursement, Health Insurance, etc.

Choose ONLY ONE category from the list:

1. Process Issues — Problems with how a benefit is applied for, claimed, or managed (e.g., reimbursement delays)
2. Coverage Issues — Complaints about what is or isn’t covered under the benefit plan
3. Benefit Value — Perceptions about whether the benefit is worth it or meets employee needs
4. Enrollment & Access Issues — Difficulty enrolling, accessing, or navigating benefit platforms
5. Communication & Clarity — Confusion due to unclear terms, poor documentation, or lack of information
6. Provider Specific Complaints — Complaints aimed at third-party providers (e.g., gym partners, HMO networks)
7. Other  — Does not clearly fit in the above categories

Respond with only the category name.

Comment: {comments}
BenefitType: {BenefitType}
BenefitSubType: {BenefitSubType}
""")

classify_chain = classify_prompt | llm | parser

# === Step 2: Action Identification ===
action_prompt = PromptTemplate.from_template("""
Given the comment,benefittype, benefitsubtype, category, sentiment, and severity, determine if this issue requires action.
If yes, summarize the action briefly in a sentence or less.

Comment: {comments}
BenefitType: {BenefitType}
BenefitSubType: {BenefitSubType}
Category: {category}
Sentiment: {sentiment}
Severity: {severity}

Return:
Actionable: Yes/No
Suggested Action: <action or 'None'>
""")
action_chain = action_prompt | llm | parser

# === Step 3: Task Routing ===
route_prompt = PromptTemplate.from_template("""
Route this issue to the correct department and assign a priority level.

Inputs:
- Category: {category}
- BenefitType: {BenefitType}
- BenefitSubType: {BenefitSubType}
- Suggested Action: {suggested_action}
- Severity: {severity}

Output format:
Department: <department>
Priority: <High/Medium/Low>
""")
route_chain = route_prompt | llm | parser


# === Step 4: Issue Summary ===
issue_summary_prompt = PromptTemplate.from_template("""
Summarize the employee's issue clearly and concisely in one sentence, using the inputs below.

Comment: {comments}
BenefitType: {BenefitType}
BenefitSubType: {BenefitSubType}
Category: {category}
Sentiment: {sentiment}
Severity: {severity}
Suggested Action: {suggested_action}

Return:
<One-sentence issue summary>
""")
issue_summary_chain = issue_summary_prompt | llm | parser


# === Sentiment Analysis using TextBlob ===
def analyze_sentiment(comment: str):
    try:
        if not comment or not isinstance(comment, str):
            return "Neutral", 3  # Default fallback

        blob = TextBlob(comment)
        polarity = blob.sentiment.polarity  # Ranges from -1 to 1
        subjectivity = blob.sentiment.subjectivity  # Ranges from 0 to 1

        # Sentiment label
        if polarity >= 0.3:
            sentiment = "Positive"
        elif polarity <= -0.3:
            sentiment = "Negative"
        else:
            sentiment = "Neutral"

        # Granular severity mapping
        if polarity >= 0.6:
            severity = 1  # Very positive
        elif polarity >= 0.3:
            severity = 2  # Mild positive
        elif polarity > -0.3:
            severity = 3  # Neutral
        elif polarity > -0.6:
            severity = 4  # Mild negative
        else:
            severity = 5  # Very negative

        # Adjust for very vague/unopinionated comments
        if subjectivity < 0.2 and sentiment == "Neutral":
            severity = 2  # Informational, not severe

        return sentiment, severity

    except Exception as e:
        print(f"[Sentiment Error]: {e}")
        return "Neutral", 3


# === Process Each Feedback Comment ===
results = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    comment = row["Comments"]
    employee_id = row["EmployeeID"]
    benefit_id = row["BenefitID"]
    BenefitType = row['BenefitType']
    BenefitSubType = row['BenefitSubType']
    # Step 1: Category Classification
    category = classify_chain.invoke({
        "comments": comment,
        "BenefitType": BenefitType,
        "BenefitSubType": BenefitSubType}).strip()

    # Step 2: Sentiment Analysis (local NLP)
    sentiment, severity = analyze_sentiment(comment)

    # Step 3: Action Identification
    action_output = action_chain.invoke({
        "comments": comment,
        "BenefitType": BenefitType,
        "BenefitSubType": BenefitSubType,
        "category": category,
        "sentiment": sentiment,
        "severity": severity
    })
    actionable, suggested_action = "No", "None"
    if "Actionable:" in action_output:
        parts = action_output.split("Suggested Action:")
        actionable = parts[0].split(":")[1].strip()
        suggested_action = parts[1].strip() if len(parts) > 1 else "None"

    # Step 4: Task Routing
    route_output = route_chain.invoke({
        "category": category,
        "BenefitType": BenefitType,
        "BenefitSubType": BenefitSubType,
        "suggested_action": suggested_action,
        "severity": severity
    })
    try:
        lines = route_output.split("\n")
        department = lines[0].split(":")[1].strip()
        priority = lines[1].split(":")[1].strip()
    except:
        department = "Unknown"
        issue_summary = "Parsing error"
        priority = "Medium"

    # Step 5: Generate Issue Summary
    issue_summary = issue_summary_chain.invoke({
        "comments": comment,
        "BenefitType": BenefitType,
        "BenefitSubType": BenefitSubType,
        "category": category,
        "sentiment": sentiment,
        "severity": severity,
        "suggested_action": suggested_action
    }).strip()

    # Add to results
    results.append({
        "EmployeeID": employee_id,
        "BenefitID": benefit_id,
        "Comments": comment,
        "BenefitType": BenefitType,
        "BenefitSubType": BenefitSubType,
        "Category": category,
        "Sentiment": sentiment,
        "Severity": severity,
        "Actionable": actionable,
        "SuggestedAction": suggested_action,
        "Department": department,
        "Priority": priority,
        "IssueSummary": issue_summary
    })

# Save final results
results_df = pd.DataFrame(results)
results_df.to_csv("feedback_chained_output.csv", index=False)
print("✅ Processed feedback saved to 'feedback_chained_output.csv'.")


100%|██████████| 25/25 [01:09<00:00,  2.79s/it]

✅ Processed feedback saved to 'feedback_chained_output.csv'.


In [7]:
results_df['Category'].unique()

array(['Benefit Value', 'Other', 'Coverage Issues', 'Process Issues',
       'Provider Specific Complaints'], dtype=object)

In [8]:
results_df['Department'].unique()

array(['Benefits Administration', 'Human Resources',
       'Life Insurance Department', 'Customer Service',
       'Health Insurance Coverage'], dtype=object)

In [9]:
results_df['SuggestedAction'].unique()

array(['None',
       'Review the tuition reimbursement process for potential improvements.',
       'Review and assess the adequacy of the vacation days offered.',
       'Review and enhance the coverage options for the supplemental standard life insurance.',
       'Simplify the reimbursement process for individual courses.',
       "Address the customer's concerns regarding service quality.",
       'Review the benefit value of the basic life insurance coverage.',
       'Review and consider increasing the time off provided.',
       'Review and enhance the value of the Tier 3 Partners gym membership.',
       'Consider enhancing the HMO Family health insurance benefits to improve customer satisfaction.',
       'Consider reviewing and enhancing the PPO Family health insurance benefits.',
       'Review and enhance the life insurance coverage options.',
       'Consider reviewing and enhancing the PPO Individual health insurance plan to improve customer satisfaction.',
       'Revie